In [7]:
# ml_api 폴더안에 api, handwriting_pics, test 폴더를 생성하고 진행한다.
# ============================================
# ml_api 폴더 구조
# ml_api/
# ├─ api/
# ├─ handwriting_pics/   ← 손글씨 이미지 저장 폴더
# └─ test/
# ============================================

import os                      # 파일/폴더 처리
import numpy as np              # 수치 연산
from PIL import Image           # 이미지 처리
import sqlite3                  # SQLite DB
from sklearn.datasets import load_digits          # digits 데이터셋
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀
from sklearn.model_selection import train_test_split # 학습/테스트 분리

# 허용할 이미지 확장자
INCLUDED_EXTENTION = [".png", ".jpg"]

# ============================================
# 1. SQLite DB 생성 및 초기화
# ============================================
dbname = 'images.db'
conn = sqlite3.connect(dbname)
cur = conn.cursor()

# cur.execute('DROP TABLE image_info')
# 처음실행시 오류가발생한다. 주석처리 이후 실행시 주석제거 후 실행
# # image_info 테이블 삭제 (없으면 오류 → IF EXISTS 권장)
cur.execute('DROP TABLE IF EXISTS image_info') 

# 이미지 파일명 저장용 테이블 생성
cur.execute('CREATE TABLE image_info(id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
conn.commit()
conn.close()

# ============================================
# 2. handwriting_pics 폴더의 이미지 파일명 DB에 저장
# ============================================
conn = sqlite3.connect(dbname)
cur = conn.cursor()

# handwriting_pics 폴더 내 파일 목록 정렬
filenames = sorted(os.listdir('handwriting_pics'))
for filename in filenames:
    base, ext = os.path.splitext(filename)  # 파일명과 확장자 분리
    if ext not in INCLUDED_EXTENTION:
        continue
    
    # 파일명을 DB에 저장
    cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
conn.commit()
cur.close()
conn.close()

# ============================================
# 3. DB에서 이미지 정보 조회
# ============================================

conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute('SELECT * FROM image_info')
pics_info = cur.fetchall()
cur.close()
conn.close()

# ============================================
# 4. 이미지 → 숫자 배열 변환 (8x8, 0~16)
# ============================================

# 예측용 이미지 데이터 저장 배열
img_test = np.empty((0, 64))  # (샘플 수, 64픽셀)
for pic_info in pics_info:
    filename = pic_info[1]
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    # 이미지 열기 → 흑백(L) 변환
    img = Image.open(f'handwriting_pics/{filename}').convert('L')
    
    # 8x8 크기로 변환 후 numpy 배열로 변환
    # sklearn digits는 밝을수록 값이 큼 → 색상 반전
    img_data256 = 255 - np.array(img.resize((8, 8)))

    # 밝기 범위 계산
    min_bright = img_data256.min()
    max_bright = img_data256.max()

    # 0~16 범위로 정규화
    img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
    img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]

# ============================================
# 5. sklearn digits 데이터로 모델 학습
# ============================================

# 기본 digits 데이터 로드 (8x8 손글씨 숫자)
digits = load_digits()
X = digits.data # (1797, 64)
y = digits.target  # 정답 레이블 (0~9)

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 로지스틱 회귀 모델 생성
logreg = LogisticRegression(max_iter=2000)

# 모델 학습
logreg_model = logreg.fit(X_train, y_train)

# ============================================
# 6. 정답(label) 생성 (파일명 기반)
# 예: 3_test.png → 정답 3
# ============================================
X_true = []
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
        
    # 파일명 맨 앞 숫자를 정답으로 사용
    X_true = X_true + [int(filename[:1])]
X_true = np.array(X_true)

# ============================================
# 7. 손글씨 이미지 예측
# ============================================
pred_logreg = logreg_model.predict(img_test)

# ============================================
# 8. 결과 출력
# ============================================
print('손글씨 문자의 판별 결과')
print('관측 결과:', X_true)
print('예측 결과:', pred_logreg)
print('정답률:', logreg_model.score(img_test, X_true))
# handwriting_pics 이미지 파일을 넣은 후 shift+enter로 실행한다.

손글씨 문자의 판별 결과
관측 결과: [0 1 2 3 4 5 6 7 8 9]
예측 결과: [1 1 4 4 4 4 4 7 4 9]
정답률: 0.4


In [8]:
# 데이터로 접근하는 코드 구현
INCLUDED_EXTENTION = [".png", ".jpg"]

# 이미지가 들어있는 폴더를 지정하고, 내용의 파일명을 취득
# images.db를 신규 작성. images.db가 이미 존재하고 있으면 접속.
dbname = 'images.db'
# 데이터베이스로의 커넥션 오브젝트 작성
conn = sqlite3.connect(dbname)
# sqlite를 조작하는 커서 오브젝트를 작성
cur = conn.cursor()
# 데이터베이스의 초기화
cur.execute('DROP TABLE image_info')
# image_info라는 table을 작성
cur.execute('CREATE TABLE image_info (id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
# 데이터베이스에 커밋하고, 변경을 보존
conn.commit()
conn.close()

# 데이터베이스에 이미지의 파일명을 삽입
conn = sqlite3.connect(dbname)
cur = conn.cursor()
filenames = sorted(os.listdir('handwriting_pics'))
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
conn.commit()
cur.close()
conn.close()

# table의 내용을 취득
conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute('SELECT * FROM image_info')
# fetchall()을 사용해서 내용을 전부 취득
pics_info = cur.fetchall()
cur.close()
conn.close()

In [9]:
# 데이터 전처리를 하는 코드
img_test = np.empty((0, 64))
#　폴더 내의 전 이미지를 데이터화
for pic_info in pics_info:
    filename = pic_info[1]
    #　이미지 파일을 취득, 그레이스케일로 하여 크기 변경
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    img = Image.open(f'handwriting_pics/{filename}').convert('L')
    img_data256 = 255 - np.array(img.resize((8, 8)))

    # 이미지 데이터 내의 최솟값이 0, 최댓값이 16이 되도록 계산
    min_bright = img_data256.min()
    max_bright = img_data256.max()
    img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
    # 가공한 이미지 데이터의 배열을 합친다
    img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]

In [10]:
# 데이터를 학습/예측/계산하는 코드

# 지도 데이터로부터의 학습
# sklearn의 데이터셋으로부터 취득, 목적 변수 X와 설명 변수 y로 나눈다
digits = load_digits()
X = digits.data
y = digits.target
# 지도 데이터와 테스트 데이터로 나눈다
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
# 로지스틱 회귀의 모델을 작성하고, 지도 데이터를 사용해서 학습시킨다
logreg = LogisticRegression(max_iter=2000)
logreg_model = logreg.fit(X_train, y_train)

# 이미지 데이터의 판별
# 이미지 데이터의 정답을 배열로 한다
X_true = []
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    X_true = X_true + [int(filename[:1])]
X_true = np.array(X_true)

# 로지스틱 회귀의 학습 완료 모델에 이미지 데이터를 넣고, 판별한다
pred_logreg = logreg_model.predict(img_test)

print('손글씨 문자의 판별 결과')
print('관측 결과:', X_true)
print('예측 결과:', pred_logreg)
print('정답률:', logreg_model.score(img_test, X_true))

손글씨 문자의 판별 결과
관측 결과: [0 1 2 3 4 5 6 7 8 9]
예측 결과: [1 1 4 4 4 4 4 7 4 9]
정답률: 0.4


In [11]:
# 1.2. 모듈 분할/ 함수 분할

# 데이터로 접근하는 코드

import os
import numpy as np
from PIL import Image
import sqlite3
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [12]:
INCLUDED_EXTENTION = [".png", ".jpg"]
dbname = 'images.db'
dir_name = 'handwriting_pics'

def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    """손글씨 문자 이미지가 놓여 있는 패스로부터 파일명을 취득하고, 리스트를 작성"""
    files = []
    filenames = sorted(os.listdir(dir_name))
    for filename in filenames:
        base, ext = os.path.splitext(filename)
        if ext not in included_ext:
            continue
        files.append(filename)
    return files

def create_table(dbname):
    """테이블을 작성하는 함수"""
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    cur.execute('DROP TABLE image_info')
    cur.execute( 'CREATE TABLE image_info (id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
    conn.commit()
    conn.close()
    print("table is successully created")

def insert_filenames(dbname, dir_name):
    """손글씨 문자 이미지의 파일명을 데이터베이스에 보존"""
    filenames = load_filenames(dir_name)
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    for filename in filenames:
        cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
    conn.commit()
    cur.close()
    conn.close()
    print("image file names are successully inserted")

def extract_filenames(dbname):
    """손글씨 문자 이미지의 파일명을 데이터베이스로부터 취득"""
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    cur.execute( 'SELECT * FROM image_info')
    filenames = cur.fetchall()
    cur.close()
    conn.close()
    return filenames

create_table(dbname)
insert_filenames(dbname, dir_name)
extract_filenames(dbname)

table is successully created
image file names are successully inserted


[(1, '0.jpg'),
 (2, '1.jpg'),
 (3, '2.jpg'),
 (4, '3.jpg'),
 (5, '4.jpg'),
 (6, '5.jpg'),
 (7, '6.jpg'),
 (8, '7.jpg'),
 (9, '8.jpg'),
 (10, '9.jpg')]

In [14]:
# 데이터의 전처리를 하는 코드

# p400, 아웃풋을 확인 필요
def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    """손글씨 문자 이미지가 놓여 있는 패스로부터 파일명을 취득하고, 리스트를 작성하는 함수"""
    files = []
    filenames = sorted(os.listdir(dir_name))
    for filename in filenames:
        base, ext = os.path.splitext(filename)
        if ext not in included_ext:
            continue
        files.append(filename)
    return files

def get_grayscale(dir_name):
    """읽어 들인 손글씨 문자 이미지의 색을 그레이스케일로 변환하는 함수(그레이스케일은 색의 농담의 명함을 나누는 방법)"""
    filenames = load_filenames(dir_name)
    for filename in filenames:
        img = Image.open(f'{dir_name}/{filename}').convert('L')
        yield img

def get_shrinked_img(dir_name):
    """이미지 크기를 8×8 픽셀의 크기로 통일하고, 밝기도 16계조의 그레이스케일로 흑백으로 변환하는 함수"""
    img_test = np.empty((0, 64))
    crop_size = 8
    for img in get_grayscale(dir_name):
        img_data256 = 255 - np.array(img.resize((crop_size, crop_size)))
        min_bright, max_bright = img_data256.min(),  img_data256.max()
        img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
        img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]
    return img_test

img_test = get_shrinked_img(dir_name)
get_shrinked_img(dir_name)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  8., 16.,  0.,  0.,  0.,  0.,  0.,
         0., 16., 16.,  8.,  0.,  0.,  0.,  0.,  0.,  8.,  8.,  8.,  0.,
         0.,  0.,  0.,  0.,  8., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,
         0.,  0.,  0.,  0.,  0.,  0.,  6.,  3.,  0.,  0.,  0.,  0.,  0.,
         3.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  3.,  0.,
         0.,  0.,  0.,  3., 16.,  9.,  6.,  0.,  0.,  0.,  

In [17]:
# 데이터를 학습/예측/계산하는 코드

# ============================================
# 필요한 라이브러리 import
# ============================================

import os                          # 파일 및 디렉터리 처리
import numpy as np                  # 수치 계산 및 배열 처리
from PIL import Image               # 이미지 파일 처리
from sklearn.datasets import load_digits          # sklearn 손글씨 숫자 데이터셋
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 모델
from sklearn.model_selection import train_test_split # 학습/검증 데이터 분리

# ============================================
# 이미지 파일 이름을 불러오는 함수
# ============================================
def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    """손글씨 문자 이미지가 놓여 있는 파일명을 취득하고, 리스트를 작성"""

    files = []                                     # 유효한 파일명을 저장할 리스트
    filenames = sorted(os.listdir(dir_name))       # 디렉터리 내 파일 목록 정렬

    for filename in filenames:
        base, ext = os.path.splitext(filename)     # 파일명과 확장자 분리

        # 지정한 확장자가 아니면 건너뜀
        if ext not in included_ext:
            continue

        files.append(filename)                     # 유효한 파일만 리스트에 추가

    return files                                   # 파일명 리스트 반환

# ============================================
# 로지스틱 회귀 모델 생성 및 학습 함수
# ============================================
def create_logreg_model():
    """로지스틱 회귀의 학습 완료 모델을 생성"""
    digits = load_digits()     # 8x8 손글씨 숫자 데이터셋 로드
    X = digits.data            # 입력 데이터 (픽셀 값, 64차원)
    y = digits.target          # 정답 레이블 (0~9)

    # 학습용 / 테스트용 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.5, random_state=0
    )

    # 로지스틱 회귀 모델 생성
    logreg = LogisticRegression(max_iter=2000)

    # 모델 학습
    logreg_model = logreg.fit(X_train, y_train)

    return logreg_model        # 학습 완료된 모델 반환

# ============================================
# 예측 결과 평가 함수
# ============================================

def evaluate_probs(dir_name, img_test, logreg_model):
    """테스트 데이터를 이용하여 로지스틱 회귀의 학습 완료 모델의 아웃풋을 평가"""
    # 디렉터리에서 파일명 로드
    filenames = load_filenames(dir_name)

    # 파일명에서 정답(label) 추출
    # 예: "3_test.png" → 3
    X_true = [int(filename[:1]) for filename in filenames]
    X_true = np.array(X_true)

    # 손글씨 이미지 예측
    pred_logreg = logreg_model.predict(img_test)

    # 결과 출력
    print('손글씨 문자의 판별 결과')
    print('관측 결과:', X_true)
    print('예측 결과:', pred_logreg)
    print('정답률:', logreg_model.score(img_test, X_true))
    return "Propability calculation is successfully finished"

# ============================================
# 실행부
# ============================================

# 로지스틱 회귀 모델 생성 및 학습
logreg_model = create_logreg_model()

# 손글씨 이미지 예측 및 평가
# dir_name : 손글씨 이미지가 저장된 폴더 경로
# img_test : 이미지 전처리 후 만든 (N, 64) 형태의 numpy 배열
evaluate_probs(dir_name, img_test, logreg_model)

손글씨 문자의 판별 결과
관측 결과: [0 1 2 3 4 5 6 7 8 9]
예측 결과: [1 1 4 4 4 4 4 7 4 9]
정답률: 0.4


'Propability calculation is successfully finished'

In [18]:
# 로지스틱 회귀의 학습 완료 모델을 생성

import pickle                                  # 파이썬 객체를 파일로 저장/로드
from sklearn.datasets import load_digits       # 손글씨 숫자 데이터셋 로드
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀 모델
from sklearn.model_selection import train_test_split # 학습/테스트 데이터 분리

# ============================================
# 1. 손글씨 숫자 데이터 로드
# ============================================
digits = load_digits()     # 8x8 크기의 손글씨 숫자 데이터셋 불러오기
X = digits.data            # 입력 데이터 (픽셀 값, shape: [샘플 수, 64])
y = digits.target          # 정답 레이블 (0~9)

# ============================================
# 2. 학습 데이터와 테스트 데이터로 분리
# ============================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,                  # 전체 데이터와 정답
    test_size=0.5,         # 절반을 테스트 데이터로 사용
    random_state=0         # 실행할 때마다 동일한 분할 결과 유지
)

# ============================================
# 3. 로지스틱 회귀 모델 생성 및 학습
# ============================================

logreg = LogisticRegression(
    max_iter=2000          # 반복 횟수를 늘려 수렴 실패 방지
)

# 학습 데이터로 모델 학습
model = logreg.fit(X_train, y_train)

# ============================================
# 4. 학습 완료된 모델을 파일로 저장 (직렬화)
# ============================================

# model.pickle 파일을 쓰기(binary) 모드로 열기
with open('model.pickle', mode='wb') as fp:
    # mode	의미	사용 예
    # 'w'	텍스트 쓰기	.txt, .csv
    # 'r'	텍스트 읽기	.txt
    # 'wb'	바이너리 쓰기	pickle, 모델, 이미지
    # 'rb'	바이너리 읽기	pickle, 모델
    # 'ab'	바이너리 추가	로그 누적
    pickle.dump(model, fp) # 학습된 모델 객체를 파일에 저장
    